TEXT

In [1]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from ProblemScripts import load_problem_KP

In [2]:
# Set random seed to ensure reproducibility
random.seed(37)

In [3]:
n_items, capacity, optimal, values, weights, items_dict = load_problem_KP('f1_l-d_kp_10_269')
# load_problem_KP('knapPI_1_100_1000_1')

number of items: 10
max weight: 269
values: [55 10 47  5  4 50  8 61 85 87]
weights: [95  4 60 32 23 72 80 62 65 46]
optimal solution: 295


In [15]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [16]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [17]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0]
724


In [4]:
def evalIndividual(individual):
    weight = 0
    value = 0
    for i in range(n_items):
        value += items_dict[i][0] * individual[i]
        weight += items_dict[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    return (value,)

def BasicEA(pop_size, n_generations):
    # Create fitness class
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    # weights is positive to signify maximisation of fitness, use negative to minimise
    # weights must be a tuple to allow to single and multi objective problems
    # to be treated in the same way

    # create individual class
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox() # Initialise DEAP toolbox
    toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_items)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    # toolbox.register("select", tools.selBest)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalIndividual)

    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    ngen, cxpb, mutpb = n_generations, 0.5, 0.2  # Number of generations, crossover probability, mutation probability

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=True)

    best_ind = hof[0]
    print(f"Best individual: {best_ind}, Value: {evalIndividual(best_ind)[0]}")


In [14]:
BasicEA(500, 10)

gen	nevals	avg  	max
0  	500   	77.86	290
1  	293   	133.738	295
2  	290   	179.12 	295
3  	281   	194.408	295
4  	307   	208.03 	295
5  	302   	231.542	295
6  	287   	241.518	295
7  	294   	259.364	295
8  	280   	276.422	295
9  	306   	281.462	295
10 	320   	285.534	295
Best individual: [0, 1, 1, 1, 0, 0, 0, 1, 1, 1], Value: 295


In [ ]:
def BasicEA_TT(pop_size, n_generations):
    # Create fitness class
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    # weights is positive to signify maximisation of fitness, use negative to minimise
    # weights must be a tuple to allow to single and multi objective problems
    # to be treated in the same way

    # create individual class
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox() # Initialise DEAP toolbox
    toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_items)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    # toolbox.register("select", tools.selBest)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalIndividual)

    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    ngen, cxpb, mutpb = n_generations, 0.5, 0.2  # Number of generations, crossover probability, mutation probability

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=True)

    best_ind = hof[0]
    print(f"Best individual: {best_ind}, Value: {evalIndividual(best_ind)[0]}")
